In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
synthetic_df = pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/Pima Diabetes/synthetic_pima_diabetic.csv')

In [11]:
x = synthetic_df.drop('Outcome',axis = 1 )
y = synthetic_df['Outcome'].to_numpy()
x = pd.get_dummies(x).iloc[:].to_numpy()

In [12]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.base import clone, BaseEstimator

try:
    import tensorflow as tf
except (ModuleNotFoundError, ImportError):
    import warnings

    warnings.warn("Tensorflow is not installed")

class ShadowModels:
    """
    Creates a swarm of shadow models and trains them with a split
    of the synthetic data.
    Parameters
    ----------
    X: ndarray or DataFrame
    y: ndarray or str
        if X it's a DataFrame then y must be the target column name,
        otherwise 
    n_models: int
        number of shadow models to build. Higher number returns
        better results but is limited by the number of records 
        in the input data.
    target_classes: int
        number of classes of the target model or lenght of the
        prediction array of the target model.
    learner: learner? #fix type
        learner to use as shadow model. It must be as similar as 
        possible to the target model. It must have `predict_proba` 
        method. Now only sklearn learners are implemented.
    Returns
    -------
    ShadowModels object
    """

    def __init__(
        self,
        X: np.ndarray,
        y: np.ndarray,
        n_models: int,
        target_classes: int,
        learner,
        **fit_kwargs,
    ) -> None:

        self.n_models = n_models
        self.X = X
        if self.X.ndim > 1:
            # flatten images or matrices inside 1rst axis
            self.X = self.X.reshape(self.X.shape[0], -1)

        self.y = y
        self.target_classes = target_classes
        self._splits = self._split_data(self.X, self.y, self.n_models, self.target_classes)
        self.learner = learner
        self.models = self._make_model_list(self.learner, self.n_models)

        # train models
        self.results = self.train_predict_shadows(**fit_kwargs)

    @staticmethod
    def _split_data(
        X: np.ndarray, y: np.ndarray, n_splits: int, n_classes: int
    ) -> List[np.ndarray]:
        """
        Split manually into n datasets maintaining class proportions
        """
        # data = np.hstack((data[0], data[1].reshape(-1, 1)))
        # X = data
        # y = data[:, -1]
        classes = range(n_classes)
        class_partitions = []
        # Split by class
        for clss in classes:

            X_clss = X[y == clss]
            y_clss = y[y == clss]
            batch_size = len(X_clss) // n_splits
            splits = []
            for i in range(n_splits):
                split_X = X_clss[i * batch_size : (i + 1) * batch_size, :]
                split_y = y_clss[i * batch_size : (i + 1) * batch_size]
                splits.append(np.hstack((split_X, split_y.reshape(-1, 1))))
            class_partitions.append(splits)

        # -------------------
        # consolidate splits into ndarrays
        # -------------------

        grouped = []
        for split in range(n_splits):
            parts = []
            for part in class_partitions:
                parts.append(part[split])
            grouped.append(parts)

        splits = []
        for group in grouped:
            splits.append(np.vstack(group))

        return splits

    @staticmethod
    def _make_model_list(learner, n) -> List:
        """
        Intances n shadow models, copies of the input parameter learner
        """
        try:
            if isinstance(learner, tf.keras.models.Model):
                models = [copy(learner) for _ in range(n)]
        except NameError:
            print("using sklearn shadow models")
            pass

        if isinstance(learner, BaseEstimator):
            models = [clone(learner) for _ in range(n)]

        return models

    def train_predict_shadows(self, **fit_kwargs):
        """
        "in" : 1
        "out" : 0
        """

        # TRAIN and predict
        results = []
        for model, data_subset in tqdm_notebook(zip(self.models, self._splits)):
            X = data_subset[:, :-1]
            y = data_subset[:, -1]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

            model.fit(X_train, y_train, **fit_kwargs)
            # data IN training set labelet 1
            y_train = y_train.reshape(-1, 1)
            predict_in = model.predict_proba(X_train)
            res_in = np.hstack((predict_in, y_train, np.ones_like(y_train)))

            # data OUT of training set, labeled 0
            y_test = y_test.reshape(-1, 1)
            predict_out = model.predict_proba(X_test)
            print(predict_out)
            res_out = np.hstack((predict_out, y_test, np.zeros_like(y_test)))

            # concat in single array
            model_results = np.vstack((res_in, res_out))
            results.append(model_results)

        results = np.vstack(results)
        return results

    def __repr__(self):
        rep = (
            f"Shadow models: {self.n_models}, {self.learner.__class__}\n"
            f"lengths of data splits : {[len(s) for s in self._splits]}"
        )
        return rep

In [13]:
logisticRegr = LogisticRegression(solver='liblinear')
sh = ShadowModels(x, y, 5, 3, logisticRegr)
shadow_data = sh.results

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:134: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

[[0.80922133 0.19077867]
 [0.22101178 0.77898822]
 [0.16537797 0.83462203]
 [0.19672692 0.80327308]
 [0.93059154 0.06940846]
 [0.8780367  0.1219633 ]
 [0.64370765 0.35629235]
 [0.8632533  0.1367467 ]
 [0.86967347 0.13032653]
 [0.52377699 0.47622301]
 [0.77534797 0.22465203]
 [0.66477924 0.33522076]
 [0.44330576 0.55669424]
 [0.89448549 0.10551451]
 [0.75158371 0.24841629]
 [0.31193622 0.68806378]
 [0.83640463 0.16359537]
 [0.76794096 0.23205904]
 [0.85481783 0.14518217]
 [0.83757534 0.16242466]
 [0.32896093 0.67103907]
 [0.40461618 0.59538382]
 [0.40676899 0.59323101]
 [0.55549319 0.44450681]
 [0.89415866 0.10584134]
 [0.2984703  0.7015297 ]
 [0.70607945 0.29392055]
 [0.67254752 0.32745248]
 [0.75108165 0.24891835]
 [0.84059358 0.15940642]
 [0.85003982 0.14996018]
 [0.74353941 0.25646059]
 [0.82160424 0.17839576]
 [0.59216314 0.40783686]
 [0.8815888  0.1184112 ]
 [0.88725741 0.11274259]
 [0.16854345 0.83145655]
 [0.19450834 0.80549166]
 [0.73094087 0.26905913]
 [0.77757074 0.22242926]


In [14]:
shadow_data

array([[0.62778544, 0.37221456, 1.        , 1.        ],
       [0.57019791, 0.42980209, 1.        , 1.        ],
       [0.58042108, 0.41957892, 0.        , 1.        ],
       ...,
       [0.97905927, 0.02094073, 0.        , 0.        ],
       [0.92738034, 0.07261966, 0.        , 0.        ],
       [0.9941775 , 0.0058225 , 1.        , 0.        ]])

In [15]:
class AttackModels:
    def __init__(self, target_classes, attack_learner):
        """
        Attacker models to learn class membership from shadow data.
        Parameters
        ----------
        target_classes: int
            number of classes that the target model can predict
        attack_learning: learner
            trainable learner to model memebership from shadow data.
            The learner its cloned into n models, one for each target class,
            and each model is trained on a class subset of the shadow data.
        Returns
        -------
        AttackModels class instance
        """
        self.target_classes = target_classes
        self.attack_learner = attack_learner
        # 1 model for each class
        self.attack_models = [clone(self.attack_learner) for _ in range(target_classes)]

        self._fited = False

    @staticmethod
    def _update_learner_params(learner, **learner_params) -> None:
        # safety check if dict is well formed
        for k in learner_params.keys():
            if not hasattr(learner, k):
                raise AttributeError(
                    f"Learner parameter {k} is not an attribute of {learner.__class__}"
                )

        # update learner params
        learner.__dict__.update(**learner_params)

    def fit(self, shadow_data, **learner_kwargs) -> None:
        """
        Trains `attack_models` with `shadow_data`. Each model is trained with
        with a subset of the same class of `shadow_data`.
        Parameters
        ----------
        shadow_data: np.ndarray
            Shadow data. Results from `ShadowModels`.
            Last column (`[:,-1]`) must be the membership label of the shadow
            prediction, where 1 means that the record was present in the 
            shadow training set ('in') and 0 if the recored was in the test
            set ('out').
            Second last column (`[:,-2]`) must be the data class. this will
            be used as grouper to split the data for each attack model.
            The rest of the columns are the class probability vector
            predicted by the shadow model.
        Returns
        -------
        None
        TODO
        ----
            Tweak model params with something like **learner_kwargs
            cross-validate
            grid search?
        """
        # split data into subsets, n == target_classes
        membership_label = shadow_data[:, -1]
        class_label = shadow_data[:, -2]
        data = shadow_data[:, :-2]
        for i, model in enumerate(self.attack_models):
            X = data[class_label == i]
            y = membership_label[class_label == i]

            # update model params
            self._update_learner_params(model, **learner_kwargs)
            # train model
            model.fit(X, y)

        self._fited = True

    def predict(self, X, y, batch=False) -> np.ndarray:
        """
        Predicts if `X` is real member of `y` in the attacked
        private training set.
        Parameters
        ----------
        X: np.ndarray
            Probability vector result from target model
        y: int, np.ndarray
            estimated class of the data record used to get `X`
        """
        if not self._fited:
            print("Must run `fit` method first")
            return

        if not batch:
            model_cls = y
            model = self.attack_models[model_cls]
            prob_vec = model.predict_proba(X)

            if y == np.argmax(prob_vec) and np.argmax(prob_vec) == 1:
                return 1

            else:
                return 0

        elif batch:

            model_classes = np.unique(y)
            res = []
            for model_cls in model_classes:
                X_cls = X[y == model_cls]
                model = self.attack_models[model_cls]
                attack_res = model.predict_proba(X_cls)
                res.append(attack_res)

            return np.concatenate(res)

In [16]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
attacker = AttackModels(target_classes=2, attack_learner=model)
attacker.fit(shadow_data)

## Membership attack on decision tree

In [17]:
df =  pd.read_csv('/content/drive/Shareddrives/CMPUT 664 : SSE/Data/Pima Diabetes/diabetes.csv')
x = df.drop('Outcome',axis = 1 )
y = df['Outcome']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.34, random_state=42)
clf = DecisionTreeClassifier(random_state=0,max_leaf_nodes = 100).fit(x_train,y_train)

In [18]:
y_pred_train = clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 100.0
accuracy on test data is 70.22900763358778


In [19]:
X_in = clf.predict_proba(x_train)
res_in = attacker.predict(X_in, y_train, batch=True)

In [20]:
X_in

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [21]:
res_in

array([[0.23, 0.77],
       [0.23, 0.77],
       [0.23, 0.77],
       ...,
       [0.93, 0.07],
       [0.93, 0.07],
       [0.93, 0.07]])

In [22]:
X_out = clf.predict_proba(x_test)
res_out = attacker.predict(X_out, y_test, batch=True)

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = np.concatenate((np.argmax(res_in, axis=1), np.argmax(res_out, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [24]:
precision_score(y_true, y_pred)

0.7180616740088106

In [25]:
recall_score(y_true, y_pred)

0.6442687747035574

In [26]:
f1_score(y_true, y_pred)

0.6791666666666667

# Membership attack on target Random Classifier

In [27]:
from sklearn.ensemble import RandomForestClassifier
r_clf = RandomForestClassifier(n_estimators = 4,random_state=0)
r_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=4, random_state=0)

In [28]:
y_pred_train = r_clf.predict(x_train)
train_a = accuracy_score(y_train, y_pred_train, normalize=True)*100
print('accuracy on train data is',train_a)

y_pred_test = r_clf.predict(x_test)
test_a = accuracy_score(y_test, y_pred_test, normalize=True)*100
print('accuracy on test data is',test_a)

accuracy on train data is 93.47826086956522
accuracy on test data is 72.51908396946564


In [29]:
X_in = r_clf.predict_proba(x_train)
res_input = attacker.predict(X_in, y_train, batch=True)

In [30]:
X_out = r_clf.predict_proba(x_test)
res_output = attacker.predict(X_out, y_test, batch=True)

In [31]:
y_pred = np.concatenate((np.argmax(res_input, axis=1), np.argmax(res_output, axis=1)))
y_true = np.concatenate((np.ones_like(y_train), np.zeros_like(y_test)))

In [32]:
precision_score(y_true, y_pred)

0.7460732984293194

In [33]:
recall_score(y_true, y_pred)

0.5632411067193676

In [34]:
f1_score(y_true, y_pred)

0.6418918918918918